# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import timeit

from datetime import datetime
from config import weather_api_key

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)
print (coordinates)

[(-39.747289238959866, 49.49291402748159), (-3.223452336123515, 47.214613294755964), (-38.729219446627816, 117.63582281070597), (-52.352896082328385, -171.21796449143474), (76.02987954064375, -42.75427218708833), (-56.35634082405106, 136.2737847376091), (8.207447889387225, -155.25455634028089), (23.922798744193173, 36.907371232562355), (36.51427856652711, 172.87610706156676), (-50.97350751825548, 7.759448004472858), (-3.7464413559980727, -94.45948888924512), (-48.747133446940325, 132.68893530342865), (-66.7338179331878, 79.48933356193379), (12.924622820498357, -95.69793727987451), (-49.45650170825761, 117.39575910802421), (54.13680065898623, 27.66393469657112), (-57.319016358396304, 1.945130939656707), (-84.02396705291017, 83.97830187046446), (-82.00438333269814, 126.88417829864142), (-60.8601090259657, -118.02820248362978), (-16.54168753714569, -111.53808183096302), (23.519596946670518, 146.43202174303656), (4.622542778570875, -88.98752895073545), (69.7693969823543, 73.6074271131505),

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [4]:
# Create an empty list for holding the cities
# Export the city_data DataFrame into a CSV file
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
         cities.append(city)
            
# Print the city count to confirm sufficient count
len(cities)

744

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [5]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [6]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=273a3bef0cd5786c0cef4bdc13ab7f5f


## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [7]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']    
            
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 2 of Set 1 | barawe
City not found. Skipping...
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | vaini
Processing Record 5 of Set 1 | ilulissat
Processing Record 6 of Set 1 | new norfolk
Processing Record 7 of Set 1 | hilo
Processing Record 8 of Set 1 | umm lajj
Processing Record 9 of Set 1 | nikolskoye
Processing Record 10 of Set 1 | cape town
Processing Record 11 of Set 1 | puerto ayora
Processing Record 12 of Set 1 | mount gambier
Processing Record 13 of Set 1 | busselton
Processing Record 14 of Set 1 | pochutla
Processing Record 15 of Set 1 | minsk
Processing Record 16 of Set 1 | rikitea
Processing Record 17 of Set 1 | katsuura
Processing Record 18 of Set 1 | yar-sale
Processing Record 19 of Set 1 | barrow
Processing Record 20 of Set 1 | tete
Processing Record 21 of Set 1 | robat karim
Processing Record 22 of Set 1 | 

Processing Record 33 of Set 4 | avera
Processing Record 34 of Set 4 | santiago del estero
Processing Record 35 of Set 4 | gravdal
Processing Record 36 of Set 4 | umm durman
City not found. Skipping...
Processing Record 37 of Set 4 | dvinskoy
Processing Record 38 of Set 4 | illoqqortoormiut
City not found. Skipping...
Processing Record 39 of Set 4 | rosetta
Processing Record 40 of Set 4 | faya
Processing Record 41 of Set 4 | coquimbo
Processing Record 42 of Set 4 | chapais
Processing Record 43 of Set 4 | khatanga
Processing Record 44 of Set 4 | sorong
Processing Record 45 of Set 4 | palmer
Processing Record 46 of Set 4 | yen bai
Processing Record 47 of Set 4 | esperance
Processing Record 48 of Set 4 | japura
Processing Record 49 of Set 4 | carasova
Processing Record 50 of Set 4 | quatre cocos
Processing Record 1 of Set 5 | makakilo city
Processing Record 2 of Set 5 | villa del rosario
Processing Record 3 of Set 5 | jining
Processing Record 4 of Set 5 | faanui
Processing Record 5 of Set 

Processing Record 24 of Set 8 | portland
Processing Record 25 of Set 8 | obo
Processing Record 26 of Set 8 | yazykovo
Processing Record 27 of Set 8 | macaboboni
City not found. Skipping...
Processing Record 28 of Set 8 | shache
Processing Record 29 of Set 8 | toribio
Processing Record 30 of Set 8 | mudgee
Processing Record 31 of Set 8 | san jose
Processing Record 32 of Set 8 | tianpeng
Processing Record 33 of Set 8 | san quintin
Processing Record 34 of Set 8 | gangotri
City not found. Skipping...
Processing Record 35 of Set 8 | hithadhoo
Processing Record 36 of Set 8 | oga
Processing Record 37 of Set 8 | iquique
Processing Record 38 of Set 8 | sherman
Processing Record 39 of Set 8 | santiago de cao
Processing Record 40 of Set 8 | ambovombe
Processing Record 41 of Set 8 | hope
Processing Record 42 of Set 8 | sioux lookout
Processing Record 43 of Set 8 | la seyne-sur-mer
Processing Record 44 of Set 8 | kurilsk
Processing Record 45 of Set 8 | tarakan
Processing Record 46 of Set 8 | consti

Processing Record 13 of Set 12 | alyangula
Processing Record 14 of Set 12 | obluche
City not found. Skipping...
Processing Record 15 of Set 12 | luebo
Processing Record 16 of Set 12 | dicabisagan
Processing Record 17 of Set 12 | nova olimpia
Processing Record 18 of Set 12 | moussoro
Processing Record 19 of Set 12 | touros
Processing Record 20 of Set 12 | karema
Processing Record 21 of Set 12 | altay
Processing Record 22 of Set 12 | raciborz
Processing Record 23 of Set 12 | ocosingo
Processing Record 24 of Set 12 | raudeberg
Processing Record 25 of Set 12 | skjervoy
Processing Record 26 of Set 12 | kauhajoki
Processing Record 27 of Set 12 | yenagoa
Processing Record 28 of Set 12 | kavaratti
Processing Record 29 of Set 12 | ugoofaaru
Processing Record 30 of Set 12 | reinosa
Processing Record 31 of Set 12 | rafaela
Processing Record 32 of Set 12 | deep river
Processing Record 33 of Set 12 | bairiki
Processing Record 34 of Set 12 | muroto
Processing Record 35 of Set 12 | swift current
Proc

In [8]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

680

## Add the weather data to a new DataFrame.

In [9]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [10]:
# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Albany,US,42.6001,-73.9662,56.34,39,99,3.69,overcast clouds
1,Vaini,TO,-21.2000,-175.2000,80.76,78,40,11.50,scattered clouds
2,Ilulissat,GL,69.2167,-51.1000,19.42,86,47,1.14,scattered clouds
3,New Norfolk,AU,-42.7826,147.0587,63.90,65,98,4.00,overcast clouds
4,Hilo,US,19.7297,-155.0900,84.25,74,75,9.22,broken clouds
5,Umm Lajj,SA,25.0213,37.2685,79.16,63,73,1.43,broken clouds
6,Nikolskoye,RU,59.7035,30.7861,37.29,60,3,14.72,clear sky
7,Cape Town,ZA,-33.9258,18.4232,59.41,62,0,19.57,clear sky
8,Puerto Ayora,EC,-0.7393,-90.3518,65.55,98,98,5.99,light rain
9,Mount Gambier,AU,-37.8333,140.7667,50.34,94,100,8.48,light rain


In [11]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Current Description'],
      dtype='object')

In [14]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_col_1= ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']

# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[new_col_1]

# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Albany,US,42.6001,-73.9662,56.34,39,99,3.69,overcast clouds
1,Vaini,TO,-21.2000,-175.2000,80.76,78,40,11.50,scattered clouds
2,Ilulissat,GL,69.2167,-51.1000,19.42,86,47,1.14,scattered clouds
3,New Norfolk,AU,-42.7826,147.0587,63.90,65,98,4.00,overcast clouds
4,Hilo,US,19.7297,-155.0900,84.25,74,75,9.22,broken clouds
5,Umm Lajj,SA,25.0213,37.2685,79.16,63,73,1.43,broken clouds
6,Nikolskoye,RU,59.7035,30.7861,37.29,60,3,14.72,clear sky
7,Cape Town,ZA,-33.9258,18.4232,59.41,62,0,19.57,clear sky
8,Puerto Ayora,EC,-0.7393,-90.3518,65.55,98,98,5.99,light rain
9,Mount Gambier,AU,-37.8333,140.7667,50.34,94,100,8.48,light rain


In [48]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [51]:
# Set the output file name
# Export the city_data DataFrame into a CSV file

output_data_file = "WeatherPy_Database.csv"

city_data_df.to_csv(output_data_file, index_label="City_ID")